# Hyperparameter tuning

In de reeds besproken machine learning technieken hebben we reeds een aantal keer vermeld dat er hyperparameters (denk aan regularisatieparameters, manieren van regularisatie, kernel type, ...).

In het geval van lineaire regressie gaat het dan over:
* L1 of L2 norm
* Regularisatieparameter $\lambda$
* learning rate

In het geval van SVM over:
* Type kernel
* Regularisatieparameter C
* Regularisatieparameter $\gamma$

Tot nu bestond de zoektocht naar de optimale combinatie van deze parameters door het manueel uitproberen en evalueren van een reeks combinaties van parameters.
Deze methode is echter niet schaalbaar en kan geautomatiseerd worden.
Dit gebeurd door middel van een gridsearch.

## Gridsearch

Het gridsearch algoritme bestaat eruit om een lijst op te stellen voor elke parameter welke waarden moeten getest worden.
Voor elke mogelijke combinatie van parameters gaat er dan een model getrained en geevalueerd worden.
Een voorbeeld van hoe dit kan geautomatiseerd worden binnen sklearn kan [hier](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) gevonden worden.

In [15]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
import numpy as np

iris = datasets.load_iris()

parameters = {
    'kernel' : ['linear', 'rbf'],
    'C' : np.arange(1,10),
    'gamma': np.arange(0.1, 2, 0.1)
}

svc = svm.SVC()
gridsearch_svc = GridSearchCV(svc, param_grid=parameters)

#%time svc.fit(iris.data, iris.target)
%time gridsearch_svc.fit(iris.data, iris.target)

gridsearch_svc.score(iris.data, iris.target)

CPU times: total: 1.31 s
Wall time: 1.34 s


0.9933333333333333

In [19]:
# heel wat informatie over veschillende uitgevoerde runs (scores en train times)
gridsearch_svc.cv_results_

# beste opvragen (niet nodig, je kan rechtstreeks het gridsearch model gebruiken)
best_svc = gridsearch_svc.best_estimator_
best_svc.score(iris.data, iris.target)

gridsearch_svc.best_params_

{'C': 2, 'gamma': 0.2, 'kernel': 'rbf'}

De standaard methode van hierboven gaat alle combinaties afgaan.
Andere methoden die sneller maar niet alle combinaties aftoetsen zijn
* [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV)
* [HalvingGridSearchCv](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html#sklearn.model_selection.HalvingGridSearchCV)
* [HalvingRandomizedSearchCv](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html#sklearn.model_selection.HalvingRandomSearchCV)

Belangrijk om hierbij op te merken is dat het GridSearch algoritme enkel verschillende parameters van het model trained en dat er geen eigenschappen van de data kan veranderd worden.
Indien je ook een exhausieve search wilt doen van het aantal hogere orde features of de vorm van scaling die gebruikt wordt op input parameters. Moet je een eigen wrapper schrijven die nog deze zaken uittest en de performantie van de uiteindelijke modellen vergelijkt.

## Validatieset

Welke data kunnen we nu gebruiken om deze gridsearch te evalueren.
Zowel de testdata als de trainingsdata kan niet gebruikt worden omdat we niet kunnen evalueren op de data waarmee het model getrained is.
Om deze reden wordt de dataset typisch in drie opgedeeld, namelijk een training-, test- en validatieset.
De validatieset is de data die dan gebruikt kan worden voor hyperparameter tuning.
Typisch wordt de dataset dan in de volgende groottes opgedeeld:

* Testset: 15%
* Validatieset: 15% 
* Trainingsdata: 70%

Dit zijn echter geen vaste waarden en kunnen wat verschillen in de praktijk.
Hoe meer data je beschibaar is hoe groter het percentage trainingsdata kan zijn. 
In het geval van big-data applicaties kan dit oplopen tot 98%.

## K-fold cross validation

Bij het steeds gebruiken van dezelfde validatieset is het mogelijk dat er een unieke split is die leidt tot een onverwacht goed of slecht resultaat.
Om dit tegen te gaan kan er gebruik gemaakt worden van K-fold cross validation.
Daarbij berekenen we de verwachte error K keer, elke keer met een andere train en validatie set om zo de kans te verhogen dat het uiteindelijke model ook goed werkt op de testset met ongeziene data.
Standaard wordt er bij het gebruik van het gridsearch algoritme gebruik gemaakt van 5 folds voor het zoeken naar de beste hyperparameters.
Indien de standaard manier niet voldoet voor de gewenste toepassing kan je ook de split rechtstreeks uitvoeren.
Meer informatie over deze methode vind je [hier](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)

![kfold cross validation](images/kFold.png)

## Oefening

In [deze dataset](https://www.kaggle.com/mathchi/diabetes-data-set) is een hele reeks data beschikbaar over een aantal medische eigenschappen van personen en of deze personen diabetes hebben of niet.
Ga nu op zoek naar het beste model om te voorspellen of een persoon diabetes gaat hebben of niet.
Test hierbij zowel de logistische regressie en svm methoden en maak gebruik van gridsearch met 10-fold cross validation om de verschillende hyperparameters te testen. 

Wat is de hoogst behaalde accuraatheid en de benodigde hyperparameters?

Indien dit gelukt is, zoek ook het model dat de hoogste weighted f1-score behaald. 
Welke techniek gebruikte dit model en welke hyperparameters zijn er hiervoor gekozen?
Vergelijk beide modellen. Is er een significant verschil in de resulterende hyperparameters?
Is de behaalde accuraatheid sterk afwijkend?

In [4]:
import opendatasets as od
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [6]:
od.download("https://www.kaggle.com/mathchi/diabetes-data-set")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading diabetes-data-set.zip to .\diabetes-data-set


100%|██████████| 8.91k/8.91k [00:00<00:00, 9.13MB/s]

In [22]:
df = pd.read_csv("./diabetes-data-set/diabetes.csv")
display(df.head())
y = df.Outcome
X = df.drop("Outcome", axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) # --> hier enkel transform gebruiken om data leakage te voorkomen

svc = SVC()
parameters = [
    {'kernel': ['linear'], 'C' : np.arange(1,10)},
    {'kernel': ['poly'], 'C' : np.arange(1,10), 'degree': np.arange(1,4)},
    {'kernel': ['rbf'], 'C' : np.arange(1,10), 'gamma': np.arange(0.1,2,0.1)}
]

gridsearch_svc = GridSearchCV(svc, parameters)
%time gridsearch_svc.fit(X_train, y_train)

gridsearch_svc.score(X_test, y_test)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


CPU times: total: 9.94 s
Wall time: 9.95 s


0.7597402597402597

## Pipeline

Bij het werken met een gridsearch is er nog een probleem omtrent data leakage.
Omdat we in bovenstaande voorbeeld de scaling hebben gedaan op de volledige dataset is alle data in principe gebruikt om de scaling uit te voeren en dus gaat er steeds een effect zijn van de training-folds op de validation-fold wat niet gewenst is.
Dit kan onterecht de score verbeteren.

Om dit tegen te gaan kan men gebruik maken van Pipelines. Hiermee definieer je de flow die de data moet ondergaan om verwerkt te worden. De volledige pipeline of flow kan dan gegeven worden aan de gridsearch algoritme. Hierdoor worden de preprocessing stappen (zoals scalers, ordinal of one-hotencoder) steeds enkel op de training folds uitgevoerd.
Dit zorgt ervoor dat er geen data leakage kan optreden. 
Het bijkomende voordeel is dat ook parameters van de preprocessing stappen meegenomen kunnen worden in de gridsearch.

Let op dat NaN waarden invullen kan gebeuren in de pipeline maar rijen of kolommen weglaten kan niet gedaan worden tijdens de pipeline. Echter is het gebruik van een pipeline een goede stap in het verhogen van de leesbaarheid van een code stuk en wordt het daardoor veelvuldig gebruikt.

In [23]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

np.random.seed(0)

X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
X

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3.0,"Zabour, Miss. Hileni",female,14.5000,1.0,0.0,2665,14.4542,None,C,None,328.0,None
1305,3.0,"Zabour, Miss. Thamine",female,NaN,1.0,0.0,2665,14.4542,None,C,None,NaN,None
1306,3.0,"Zakarian, Mr. Mapriededer",male,26.5000,0.0,0.0,2656,7.2250,None,C,None,304.0,None
1307,3.0,"Zakarian, Mr. Ortin",male,27.0000,0.0,0.0,2670,7.2250,None,C,None,NaN,None


In [27]:
# preprocessing
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

numeric_features = ["age", "fare"]
categoric_features = ["sex", 'pclass', "embarked"]

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="mean")),    # steeds naam en object
    ('scaler', StandardScaler())
])

categoric_transformer = OneHotEncoder(handle_unknown='ignore')

# anders behandelen van numerieke en categorieke data
preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_features),                 # naam - object dat het moet uitvoeren - lijst met kolommen waarop het moet uitgevoerd worden
    ("cat", categoric_transformer, categoric_features)
])          # remainder parameter -> default drop (laat kolommen niet gespecifieerd wegvallen)

# ML gedeelte
classifier = Pipeline([
    ('preprocessor', preprocessor),
    ('clf', SVC())
])

#classifier.fit(X_train, y_train)
#classifier.score(X_test, y_test)

In [33]:
parameters = [
    {'preprocessor__num__imputer__strategy': ["median", "mean"], 'clf__kernel': ['linear'], 'clf__C' : np.arange(1,10)},
    {'clf__kernel': ['poly'], 'clf__C' : np.arange(1,10), 'clf__degree': np.arange(1,4)},
    {'clf__kernel': ['rbf'], 'clf__C' : np.arange(1,10), 'clf__gamma': np.arange(0.1,2,0.1)}
]     
# met pipelines kan je hier speciifieren welke parameters van welke componenten in de pipeline aangepast moeten worden.
# met dubbele underscores __ kan je een niveau afdalen
# in de parameter grid worden altijd lijstjes verwacht (itereerbaar) en geen enkele waarden

grid = GridSearchCV(classifier, parameters)
%time grid.fit(X_train, y_train)

grid.score(X_test, y_test)

CPU times: total: 33 s
Wall time: 33 s


0.8320610687022901

## Oefening

Maak een pipeline voor een gridsearch uit te voeren op de diabetes-dataset in sklearn. Gebruik een Min-Max scaler voor de numerieke waarden en een ordinal encoder voor de categorieke kolommen.
Zoek naar de beste combinatie van hyperparameters van een SVM-classifier door middel van een grid search met cross validation.